In [1]:
import asyncio
from importlib.resources import read_text
from collections import Counter
from time import sleep

import pandas as pd
from pprint import pprint
import simplejson as json
from tqdm import tqdm

from rest_api.db import select
import toloka.configs as configs
from toloka.interface import Toloka
from toloka.task import Task
from toloka.operation import Operation
from toloka.pool import Pool
from toloka.assignments import Assignment


async with Toloka.init_session() as session:
    res = await Assignment.list(pool_id="248637")
res = [list(zip(x['solutions'], x['tasks'])) for x in res['items'] if x['status'] == 'ACCEPTED']
res = [(y[0]['output_values'], y[1]['input_values']) for x in res for y in x]
df_results = pd.DataFrame.from_records(
    [(x['doubt'], x['error'], x['result'],
      int(y['image_left'].split('.')[0]),
      int(y['image_right'].split('.')[0]))
     for x, y in res],
    columns=['Doubt', 'Error', 'Result', 'EventId1', 'EventId2']
)
res, cols = select("""
SELECT
    EventId1,
    EventId2,
    m.Confidence,
    f1.Age Age1,
    f2.Age Age2,
    f1.Gender Gender1,
    f2.Gender Gender2,
    f1.Quality Quality1,
    f2.Quality Quality2
FROM (
    SELECT DISTINCT
        IIF(m.EventId1 > m.EventId2, m.EventId1, m.EventId2) EventId1,
        IIF(m.EventId1 > m.EventId2, m.EventId2, m.EventId1) EventId2,
        m.Confidence
    FROM FFMatrix m WITH(nolock)
        JOIN FindFaceEvents f1 WITH (nolock) ON f1.Id=EventId1 AND f1.Store LIKE '107%' AND f1.CreatedDate BETWEEN '2019-09-11' AND '2019-09-13'
        JOIN FindFaceEvents f2 WITH (nolock) ON f2.Id=EventId2 AND f2.Store LIKE '107%' AND f2.CreatedDate BETWEEN '2019-09-11' AND '2019-09-13'
    WHERE m.Confidence > 0.67 AND EventId1 <> EventId2

) m
    JOIN FindFaceEvents f1 WITH (nolock) ON f1.Id=EventId1 AND f1.Store LIKE '107%' AND f1.CreatedDate BETWEEN '2019-09-11' AND '2019-09-13'
    JOIN FindFaceEvents f2 WITH (nolock) ON f2.Id=EventId2 AND f2.Store LIKE '107%' AND f2.CreatedDate BETWEEN '2019-09-11' AND '2019-09-13'
""")
df_conf = pd.DataFrame.from_records(res, columns=cols)
df = pd.merge(df_results, df_conf, on=['EventId1', 'EventId2'], how="inner")
df['EventId1'] = df['EventId1'].astype(str)
df['EventId2'] = df['EventId2'].astype(str)
df['Age1'] = df['Age1'].astype(float)
df['Age2'] = df['Age2'].astype(float)
df['Quality1'] = df['Quality1'].astype(float)
df['Quality2'] = df['Quality2'].astype(float)
df['Confidence'] = df['Confidence'].astype(float)
df.to_excel('C:/Users/Somov.A/Desktop/toloka_results.xlsx', index=False)


In [ ]:
import asyncio

async def main():
    await asyncio.sleep(2)
    return 1

await main()